<a href="https://colab.research.google.com/github/hsieh2000/HW/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ckiptagger
!pip install ckip_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 115.1 MB/s eta 0:00:00


In [3]:
import os
import re
import numpy as np
import pandas as pd
import jieba
import jieba.posseg as pseg
from ckiptagger import WS, POS, NER

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [4]:
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker
# Initialize drivers
print("Initializing drivers ... WS")
ws_driver = CkipWordSegmenter(model="bert-base", device=0)
print("Initializing drivers ... POS")
pos_driver = CkipPosTagger(model="bert-base", device=0)
print("Initializing drivers ... NER")
ner_driver = CkipNerChunker(model="bert-base", device=0)
print("Initializing drivers ... all done")
print()

Initializing drivers ... WS


Initializing drivers ... POS


Initializing drivers ... NER


Initializing drivers ... all done



In [5]:
os.chdir("/content/drive/MyDrive/NLP")

In [6]:
ORIGINAL_DATA = pd.read_csv("data.csv")
ORIGINAL_DATA = ORIGINAL_DATA.drop(columns = ["Unnamed: 0"])
content = ORIGINAL_DATA["content"]


In [7]:
ORIGINAL_DATA

,article_title,content,message_count_all,message_count_boo,message_count_count,message_count_neutral,message_count_push
0,[狂幹] 孫運璿 vs 陳水扁,孫前院長和陳水扁的比較： 出身貧窮人家，苦讀完成學業，非經濟科班出身，任公職後，卻能為國拼出...,233,49,58,77,107
1,[無言] 國王的新衣,變調的童話：國王的新衣 他能在眾王子的爭奪戰中脫穎而出，除了自身的實力 其實也背負著全國極欲...,85,7,32,39,39
2,[吐嘈] =====沈老大在53台，婊DPP的4個影片片段=====,2006/05/17 民視新聞台 頭家來開講 婊DPP的4個clips 每一段都是切重要害的...,47,3,27,14,30
3,[大話] 笑話新聞 徐議員批馬片段,其實不只有徐議員啦 其他的人的講法 也大多都有入鏡 =================分格線...,21,1,11,8,12
4,[轉錄][囧rz]安西教練：「三井，你負責投外線！」,作者: mushishi (蟲師) 看板: joke 標題: [囧rz]安西教練：三井，你負...,150,3,134,10,137
...,...,...,...,...,...,...,...
79798,[討論] 這是第一次中央民調沒崩但地方大敗的局面嗎？,有長期看板上分享的民調就可以知道 不管總統或政院的滿意度 都沒像2006、2014或2018...,0,0,0,0,0
79799,[公告] 政治黑特板板規__20220803,第一章 總則 1.本板接受中華民國政府頒布之法律，教育部學術網路公約，以及PTT站方制定之...,0,0,0,0,0
79800,[公告] HatePolitic 發文門檻變更,選舉期間 發文門檻調到2000次 選舉過後視時機調回原次數 組務那邊也申請很久才設定好 也沒...,70,1,42,26,43
79801,[討論] 11月16日0時起不得再發布 評論相關民調,《公職人員選舉罷免法》第53條第2項規定，投票前10天起，也就是 11月16日0時起 ， 不...,39,0,13,26,13


In [36]:
def read_stopword():
    with open("./stopwords_TW.txt", "r", encoding="utf-8") as f:
        stopword = [word.strip("\n") for word in f.readlines()]
    return stopword
stopwords = read_stopword()

In [9]:
# def remove_stopword(word_seg, stopword):
#     word_lst = []    
#     for word in word_seg:
#       if word not in stopword:
#           word_lst.append(word)
#     sentance = "".join(word_lst)
#     return sentance


In [10]:
# ws_results = ws_driver([content[0]])
# pos_results = pos_driver([content[0]])
# ner_results = ner_driver([content[0]])
# print(ws_results[0])
# print(pos_results[0])
# print(ner_results[0])



Inference: 100%|██████████| 1/1 [00:00<00:00, 30.61it/s]

['孫', '前院長', '和', '陳水扁', '的', '比較', '：', ' ', '出身', '貧窮', '人家', '，', '苦讀', '完成', '學業', '，', '非', '經濟', '科班', '出身', '，', '任', '公職', '後', '，', '卻', '能', '為', '國', '拼出', '亮麗', '的', '經濟', '成果', ' ', '出身', '貧窮', '人家', '，', '苦讀', '完成', '學業', '，', '非', '經濟', '科班', '出身', '，', '任', '公職', '後', '，', '卻', '能', '為', '家', '拼出', '亮麗', '的', '經濟', '成果', ' ', '由', '山東', '到', '台灣', '，', '不', '喊', '愛', '台灣', '，', '帶領', '台灣', '由', '破敗', '走向', '繁榮', ' ', '由', '台南', '到', '台北', '，', '常', '喊', '愛', '台灣', '，', '帶領', '台灣', '由', '繁榮', '走向', '破敗', ' ', '根據', '傳記', '，', '兒子', '當兵', '時', '國防部長', '不', '知道', ' ', '根據', '報導', '，', '兒子', '當兵', '是', '開', '積架車', '上下班', ' ', '一', '個', '幫', '工商界', '解決', '他們', '的', '經營', '問題', ' ', '一', '個', '要', '工商界', '解決', '自己', '的', '經濟', '問題', ' ', '瞭解', '關於', '他', '的', '事蹟', '，', '心情', '感動', '到', '飆淚', ' ', '瞭解', '關於', '他', '的', '事蹟', '，', '心情', '激動', '到', '開罵', ' ', '原來', '，', '風範', '就', '在', '這些', '地方', '展現', '...', '..', '.', ' ', '不能', '再', '比', '了', '，', '再', '比', '會', '玷污', '了', 

In [38]:
text = "經過多年激烈戰事，複製人大戰即將結束。絶地議會派歐比王將導致戰亂的主謀者繩之以法；不料，西斯勢力已悄悄深入銀河系，勢力漸大的議長白卜庭用黑暗勢力的力量，誘惑天行者安納金轉變成黑武士達斯維達，幫助他達成心願建立銀河帝國，剷除絕地武士…【星際大戰】系列電影最後一塊拼圖，喬治盧卡斯不僅要解開黑武士的影壇跨世紀謎團，更要著手打造影史最大星際戰爭。"

In [39]:
def do_CKIP_WS(article):
  ws_results = ws_driver([article])
  return ws_results


In [40]:
def do_CKIP_POS(ws_result):
    # write your code in here
    pos = pos_driver(ws_result[0])
    all_list = []
    for sent in pos:

      all_list.append(sent)
    return all_list
 

In [55]:
def cleaner(ws_results, pos_results, stopwords):
  for ws, pos in zip(ws_results[0], pos_results):
    in_stopwords_or_not = ws not in stopwords
    if_len_greater_than_1 = len(ws) > 1
    if in_stopwords_or_not and if_len_greater_than_1:
      print(ws)
    else:
      pass

In [56]:
ws_results = do_CKIP_WS(text)
pos_result = do_CKIP_POS(ws_results)

Inference: 100%|██████████| 1/1 [00:00<00:00, 20.29it/s]


In [57]:
ans = cleaner(ws_results, pos_result, stopwords)

激烈
戰事
複製人
大戰
結束
絶地
議會
歐比王
導致
戰亂
主謀
繩之以法
西斯
勢力
悄悄
深入
銀河系
勢力
議長
白卜庭
黑暗
勢力
力量
誘惑
天行者
安納金
轉變成
黑武士
達斯維達
幫助
達成
心願
建立
銀河
帝國
剷除
絕地
武士
星際
大戰
系列
電影
最後
拼圖
喬治
盧卡斯
解開
黑武士
影壇
世紀
謎團
著手
打造
影史
星際
戰爭


In [49]:
test = ["你好","我叫","Jeremy"]
"你" in test

False

In [16]:
ws_results = do_CKIP_WS(content[0])
pos_result = do_CKIP_POS(ws_results)


Inference: 100%|██████████| 1/1 [00:00<00:00,  8.72it/s]


In [18]:
len(pos_result)

184

In [ ]:
def do_CKIP_NER(data):
    # write your code in here
    C_sents_annotated_ner=ner_driver(data)
    all_list = []
    ner_filter=['PERSON','ORG','GPE','MONEY']
    for sent in C_sents_annotated_ner:
      filter_ner_sent=[]
      for j,word in enumerate(sent):
          if word.ner in ner_filter:
              filter_ner_sent.append(word.word)
      all_list.append(filter_ner_sent)
    return all_list
 

In [ ]:
y = do_CKIP_NER(ws_results[0])
y

Inference: 100%|██████████| 1/1 [00:00<00:00,  8.76it/s]


[[],
 [],
 [],
 ['陳水扁'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['山東'],
 [],
 ['台灣'],
 [],
 [],
 [],
 [],
 ['台灣'],
 [],
 [],
 ['台灣'],
 [],
 [],
 [],
 [],
 [],
 [],
 ['台南'],
 [],
 ['台北'],
 [],
 [],
 [],
 [],
 ['台灣'],
 [],
 [],
 ['台灣'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [ ]:
  for word_ws, word_pos in zip(sentence_ws, sentence_pos):
    # 只留名詞和動詞
    is_N_or_V = word_pos.startswith("V") or word_pos.startswith("N")
    # 去掉名詞裡的某些詞性
    is_not_stop_pos = word_pos not in stop_pos
    # 只剩一個字的詞也不留
    is_not_one_charactor = not (len(word_ws) == 1)
    # 組成串列
    if is_N_or_V and is_not_stop_pos and is_not_one_charactor:
      short_with_pos.append(f"{word_ws}({word_pos})")
      short_sentence.append(f"{word_ws}")